In [ ]:
%pip install pulp

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 24.4 MB/s eta 0:00:0000:0100:01


In [9]:
import pulp as pl
import numpy as np
from scipy.stats import multivariate_t

print("==============================================================================")

# Define the problem
model = pl.LpProblem("Maximize_Profit", pl.LpMaximize)

# Products, machines and days
subproducts = ['Ciasto_na_chleb_domowy_0_5kg', 'Ciasto_na_chleb_domowy_0_9kg', 'Ciasto_na_chleb_na_zakwasie', 'Ciasto_na_bulki', 'Ciasto_na_bagietki', 'Ciasto_na_rogaliki_z_czekolada']
products = ['Chleb_domowy_0_5kg', 'Chleb_domowy_0_9kg', 'Chleb_na_zakwasie', 'Bulki', 'Bagietki', 'Rogaliki_z_czekolada']
days = ['Pon', 'Wt', 'Sr', 'Czw', 'Pt', 'Sob']
machines = {'Dzieza', 'Wazenie', 'Dzielarka', 'Tabula', 'Rogalikarka', 'Nadziewarka', 'Garownia', 'Piec', 'Pomieszczenie_do_studzenia', 'Pakowanie'}

# Income from product sales and production cost in PLN/piece
product_income = {'Chleb_domowy_0_5kg': 5.2, 'Chleb_domowy_0_9kg': 7.9, 'Chleb_na_zakwasie': 6.2, 'Bulki': 1.4, 'Bagietki': 1.55, 'Rogaliki_z_czekolada': 2.1}
sub_product_cost = {'Ciasto_na_chleb_domowy_0_5kg': 1.6, 'Ciasto_na_chleb_domowy_0_9kg': 2.0, 'Ciasto_na_chleb_na_zakwasie': 2.0, 'Ciasto_na_bulki': 0.5, 'Ciasto_na_bagietki': 0.6, 'Ciasto_na_rogaliki_z_czekolada': 1.0}
product_cost = {'Chleb_domowy_0_5kg': 0.8, 'Chleb_domowy_0_9kg': 0.9, 'Chleb_na_zakwasie': 0.8, 'Bulki': 0.2, 'Bagietki': 0.2, 'Rogaliki_z_czekolada': 0.2}

# Time available
shifts_per_day = 2
hours_per_shift = 8
hours_per_day = shifts_per_day * hours_per_shift

# Initial stock and storage cost
initial_stock = 0
fridge_count = 5
fridge_capacity = 160
end_stock = 0
storage_capacity = fridge_count * fridge_capacity

# Machines cost
fridge_cost_per_day = 0.281 * 24 * 1.4 * 0.2 # kW * 24h * price per hour * factor
furnace_cost_per_hour = 82.4 # https://giko.pl/kalkulator-zuzycia-energii-dla-piecow-piekarniczych/

# Aux constants
max_batches_count = 4
large_number = 100000

# Production times required per product in hours
production_times = {
    'Ciasto_na_chleb_domowy_0_5kg': {'Dzieza': 4.7, 'Wazenie': 0.29, 'Dzielarka': 0.0, 'Tabula': 0.7, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.0, 'Piec': 0.0, 'Pomieszczenie_do_studzenia': 0.0, 'Pakowanie': 0.0},
    'Ciasto_na_chleb_domowy_0_9kg': {'Dzieza': 5, 'Wazenie': 0.2, 'Dzielarka': 0.0, 'Tabula': 0.55, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.0, 'Piec': 0.0, 'Pomieszczenie_do_studzenia': 0.0, 'Pakowanie': 0.0},
    'Ciasto_na_chleb_na_zakwasie': {'Dzieza': 8.6, 'Wazenie': 0.22, 'Dzielarka': 0.0, 'Tabula': 0.6, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.0, 'Piec': 0.0, 'Pomieszczenie_do_studzenia': 0.0, 'Pakowanie': 0.0},
    'Ciasto_na_bulki': {'Dzieza': 1, 'Wazenie': 0.0, 'Dzielarka': 0.23, 'Tabula': 0.34, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.0, 'Piec': 0.0, 'Pomieszczenie_do_studzenia': 0.0, 'Pakowanie': 0.0},
    'Ciasto_na_bagietki': {'Dzieza': 0.95, 'Wazenie': 0.0, 'Dzielarka': 0.24, 'Tabula': 0.0, 'Rogalikarka': 0.28, 'Nadziewarka': 0.0, 'Garownia': 0.0, 'Piec': 0.0, 'Pomieszczenie_do_studzenia': 0.0, 'Pakowanie': 0.0},
    'Ciasto_na_rogaliki_z_czekolada': {'Dzieza': 0.55, 'Wazenie': 0.0, 'Dzielarka': 0.28, 'Tabula': 0.0, 'Rogalikarka': 0.31, 'Nadziewarka': 0.4, 'Garownia': 0.0, 'Piec': 0.18, 'Pomieszczenie_do_studzenia': 0.0, 'Pakowanie': 0.0},
    'Chleb_domowy_0_5kg': {'Dzieza': 0.0, 'Wazenie': 0.0, 'Dzielarka': 0.0, 'Tabula': 0.0, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.0, 'Piec': 0.68, 'Pomieszczenie_do_studzenia': 1.8, 'Pakowanie': 0.56},
    'Chleb_domowy_0_9kg': {'Dzieza': 0.0, 'Wazenie': 0.0, 'Dzielarka': 0.0, 'Tabula': 0.0, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.5, 'Piec': 1.2, 'Pomieszczenie_do_studzenia': 2.5, 'Pakowanie': 0.57},
    'Chleb_na_zakwasie': {'Dzieza': 0.0, 'Wazenie': 0.0, 'Dzielarka': 0.0, 'Tabula': 0.0, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.4, 'Piec': 1.0, 'Pomieszczenie_do_studzenia': 2.3, 'Pakowanie': 0.6},
    'Bulki': {'Dzieza': 0.0, 'Wazenie': 0.0, 'Dzielarka': 0.0, 'Tabula': 0.0, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.38, 'Piec': 0.25, 'Pomieszczenie_do_studzenia': 0.48, 'Pakowanie': 0.0},
    'Bagietki': {'Dzieza': 0.0, 'Wazenie': 0.0, 'Dzielarka': 0.0, 'Tabula': 0.0, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.33, 'Piec': 0.23, 'Pomieszczenie_do_studzenia': 0.42, 'Pakowanie': 0.0},
    'Rogaliki_z_czekolada': {'Dzieza': 0.0, 'Wazenie': 0.0, 'Dzielarka': 0.0, 'Tabula': 0.0, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.25, 'Piec': 0.18, 'Pomieszczenie_do_studzenia': 0.82, 'Pakowanie': 0.42}
}

# Market limits for sales of Products
# market_limits = {
#     "Chleb_domowy_0_5kg": {"Pon": 200, "Wt": 30, "Sr": 0, "Czw": 20, "Pt": 30, "Sob": 18},
#     "Chleb_domowy_0_9kg": {"Pon": 150, "Wt": 25, "Sr": 80, "Czw": 150, "Pt": 250, "Sob": 180},
#     "Chleb_na_zakwasie": {"Pon": 180, "Wt": 280, "Sr": 0, "Czw": 180, "Pt": 280, "Sob": 18},
#     "Bulki": {"Pon": 400, "Wt": 500, "Sr": 0, "Czw": 40, "Pt": 50, "Sob": 180},
#     "Bagietki": {"Pon": 350, "Wt": 450, "Sr": 0, "Czw": 350, "Pt": 45, "Sob": 180},
#     "Rogaliki_z_czekolada": {"Pon": 300, "Wt": 400, "Sr": 0, "Czw": 300, "Pt": 400, "Sob": 180},
# }
market_limits = {
    "Chleb_domowy_0_5kg": {"Pon": 290, "Wt": 530, "Sr": 240, "Czw": 420, "Pt": 530, "Sob": 680},
    "Chleb_domowy_0_9kg": {"Pon": 350, "Wt": 525, "Sr": 380, "Czw": 150, "Pt": 250, "Sob": 580},
    "Chleb_na_zakwasie": {"Pon": 480, "Wt": 280, "Sr": 320, "Czw": 180, "Pt": 280, "Sob": 418},
    "Bulki": {"Pon": 620, "Wt": 670, "Sr": 840, "Czw": 530, "Pt": 650, "Sob": 580},
    "Bagietki": {"Pon": 350, "Wt": 750, "Sr": 430, "Czw": 350, "Pt": 650, "Sob": 680},
    "Rogaliki_z_czekolada": {"Pon": 300, "Wt": 400, "Sr": 240, "Czw": 310, "Pt": 400, "Sob": 280},
}

# Maximum batch size
batch_size = {
    "Ciasto_na_chleb_domowy_0_5kg": 240,
    "Ciasto_na_chleb_domowy_0_9kg": 130,
    "Ciasto_na_chleb_na_zakwasie": 200,
    "Ciasto_na_bulki": 200,
    "Ciasto_na_bagietki": 200,
    "Ciasto_na_rogaliki_z_czekolada": 200,
    "Chleb_domowy_0_5kg": 240,
    "Chleb_domowy_0_9kg": 130,
    "Chleb_na_zakwasie": 200,
    "Bulki": 200,
    "Bagietki": 200,
    "Rogaliki_z_czekolada": 200,
}


# Machines available
machines_available = {"Dzieza": 6, "Wazenie": 2, "Dzielarka": 2, "Tabula": 3, "Rogalikarka": 1, "Nadziewarka": 1, "Garownia": 2, "Piec": 40, "Pomieszczenie_do_studzenia": 4, "Pakowanie": 2}

# Variables for production, sales, and storage of products each day
sub_production_vars = pl.LpVariable.dicts("Subproduct production", ((subproduct, day) for subproduct in subproducts for day in days), lowBound=0, cat='Integer')
production_vars = pl.LpVariable.dicts("Production", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
batch_count_vars = pl.LpVariable.dicts("Batches Count", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
subbatch_count_vars = pl.LpVariable.dicts("Subproduct Batches Count", ((subproduct, day) for subproduct in subproducts for day in days), lowBound=0, cat='Integer')
product_start_time = pl.LpVariable.dicts("Production start time for batch", ((product, machine, day, batch) for product in products for day in days for machine in machines for batch in range(max_batches_count)), lowBound=0)
subproduct_start_time = pl.LpVariable.dicts("Subproduction start time for batch", ((subproduct, machine, day, batch) for subproduct in subproducts for day in days for machine in machines for batch in range(max_batches_count)), lowBound=0)
sales_vars = pl.LpVariable.dicts("Sales", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
storage_vars = pl.LpVariable.dicts("Storage", ((subproduct, day) for subproduct in subproducts for day in days), lowBound=0, upBound=storage_capacity, cat='Integer')
fridge_count_usage_vars = pl.LpVariable.dicts("FridgeCount", ((day) for day in days), lowBound=0, upBound=fridge_count, cat='Integer')

# Inventory and sales constraints
for subproduct, product in zip(subproducts, products):
    model += storage_vars[subproduct, 'Pon'] == initial_stock + sub_production_vars[subproduct, 'Pon'] - production_vars[product, 'Pon']
    for i in range(1, len(days)):
        day = days[i]
        prev_day = days[i-1]
        model += storage_vars[subproduct, day] == storage_vars[subproduct, prev_day] + sub_production_vars[subproduct, day] - production_vars[product, day]
    # model += storage_vars[product, 'Sob'] == end_stock  # Ending stock

# Machine time constraints for each product and day
for day in days:
    # total_production_time = sum((production_vars[product, day] * production_times[product][machine]) / (machines_available[machine] * batch_size[product])
    #                             for product in products
    #                             for machine in machines) \
    #                         + sum((sub_production_vars[subproduct, day] * production_times[subproduct][machine]) / (machines_available[machine] * batch_size[subproduct])
    #                             for subproduct in subproducts
    #                             for machine in machines)
    # model += total_production_time <= hours_per_day, f"Global_production_time_{day}"
    model += storage_capacity >= sum(storage_vars[subproduct, day] for subproduct in subproducts for day in days)
    model += fridge_count_usage_vars[day] * fridge_capacity >= sum(storage_vars[subproduct, day] for subproduct in subproducts)
    # model += sales_vars['Bagietki',day] + sales_vars['Bulki',day] <= 8 * sales_vars['Rogaliki_z_czekolada', day]
    # model += sales_vars['Bagietki',day] >= 0.2 * (sales_vars['Chleb_domowy_0_5kg', day] + sales_vars['Chleb_na_zakwasie', day])
    # model += sales_vars['Bulki',day] >= 0.05 * (sales_vars['Chleb_domowy_0_9kg', day])

# Market sales constraints
for product in products:
    # model += sales_vars[product, 'Pon'] + sales_vars[product, 'Wt'] >= 0.5 * (market_limits[product]['Pon'] + market_limits[product]['Wt'])
    # model += sales_vars[product, 'Sr'] + sales_vars[product, 'Czw'] >= 0.5 * (market_limits[product]['Sr'] + market_limits[product]['Czw'])
    # model += sales_vars[product, 'Pt'] + sales_vars[product, 'Sob'] >= 0.5 * (market_limits[product]['Pt'] + market_limits[product]['Sob'])
    for day in days:
        model += sales_vars[product, day] == production_vars[product, day]
        model += sales_vars[product, day] <= market_limits[product][day], f"Max_sales_{product}_{day}"
        # model += sales_vars[product, day] <= production_vars[product, day] + (storage_vars[product, days[days.index(day) - 1]] if day != 'Pon' else initial_stock)
        model += batch_count_vars[product, day] * batch_size[product] >= production_vars[product, day]
        # model += sales_vars[product, day] >= 0.5 * market_limits[product][day]
        # model += sales_vars[product, day] >= 0.05 * market_limits[product][day]

machines = list(machines)
for subproduct in subproducts:
    for day in days:
        for i in range(1, 6): # 5 is Nadziewarka
            prev = 1
            machine = machines[i]
            prev_machine = machines[i-prev]

            while production_times[subproduct][prev_machine] == 0.0:
                prev += 1
                prev_machine = machines[i-prev]

            for batch in range(max_batches_count):
                model += subproduct_start_time[subproduct, machine, day, batch] >= subproduct_start_time[subproduct, prev_machine, day, batch] + production_times[subproduct][prev_machine]
                model += subproduct_start_time[subproduct, machine, day, batch] <= (subbatch_count_vars[subproduct, day] - batch) * large_number
                
        model += hours_per_day >= subproduct_start_time[subproduct, machines[5], day, batch] + production_times[subproduct][machines[5]]


for product in products:
    for day in days:
        for i in range(7, len(machines)):
            prev = 1
            machine = machines[i]
            prev_machine = machines[i-prev]

            while production_times[product][prev_machine] == 0.0:
                prev += 1
                prev_machine = machines[i-prev]

            for batch in range(max_batches_count):
                model += product_start_time[product, machine, day, batch] >= product_start_time[product, prev_machine, day, batch] + production_times[product][prev_machine]
                model += product_start_time[product, machine, day, batch] <= (batch_count_vars[product, day] - batch) * large_number
                
        model += hours_per_day >= product_start_time[product, machines[-1], day, batch] + production_times[product][machines[-1]]

machines = set(machines)


# Objective function: Maximize profit from sales minus costs
model += pl.lpSum([sales_vars[product, day] * product_income[product] for product in products for day in days]) \
                    - pl.lpSum([production_vars[product, day] * product_cost[product] for product in products for day in days]) \
                    - pl.lpSum([sub_production_vars[subproduct, day] * sub_product_cost[subproduct] for subproduct in subproducts for day in days]) \
                    - pl.lpSum([fridge_count_usage_vars[day] * fridge_cost_per_day for day in days]) \
                    - pl.lpSum([batch_count_vars[product, day] * production_times[product]['Piec'] * furnace_cost_per_hour for product in products for day in days]), "Total_Profit"

solver = pl.PULP_CBC_CMD(timeLimit=20)

model.solve(solver)

# model.solve()

# Results
print("Status:", pl.LpStatus[model.status])
if pl.LpStatus[model.status] == 'Optimal':
    print("Total Profit:", pl.value(model.objective))

    print(f"{'Product':<10} {'day':<10} {'Production':<12} {'Sales':<10} {'Storage':<10}")

    for product, subproduct in zip(products, subproducts):
        for day in days:
            subproduction = sub_production_vars[subproduct, day].varValue
            production = production_vars[product, day].varValue
            sales = sales_vars[product, day].varValue
            storage = storage_vars[subproduct, day].varValue
            print(f"{product:<10} {day:<10} {subproduction:<12} {production:<12} {sales:<10} {storage:<10}")

    # print("\nProduction Time Usage per Product and Day:")
    # for day in days:
    #     print(f"\Day: {day}")
    #     for product in products:
    #         total_production_time = sum(
    #             batch_count_vars[product, day].varValue * production_times[product]s.get(machine, 0) 
    #             for machine in machines #.keys()
    #         )
    #         print(f"  Product: {product}, Total Production Time: {total_production_time:.2f} hours")

else:
    print("No optimal solution was found.")
    # Analiza ograniczeń
    for name, constraint in model.constraints.items():
        if constraint.pi < 0:
            print(f"Ograniczenie {name}: {constraint}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gromiusz/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/645c448a294b4b2f939ff5c885f8bbf3-pulp.mps -max -sec 20 -timeMode elapsed -branch -printingOptions all -solution /tmp/645c448a294b4b2f939ff5c885f8bbf3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2537 COLUMNS
At line 8388 RHS
At line 10921 BOUNDS
At line 11144 ENDATA
Problem MODEL has 2532 rows, 1638 columns and 5256 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 20
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.01 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.02   (Wallclock seconds):       0.01

Status: Infeasible
No optimal solution was found.
Ograniczenie _C64: 130*Batches_Count_('Chleb_domowy_0_9kg',_'Wt') - Production_('Chleb_domowy_0_9kg',_'Wt') >= 0
Ograniczen